<a href="https://colab.research.google.com/github/TaraRasti/Deep_Learning_F20_Assignments/blob/master/Assignment%208/dropout/FashionMNIST_SGD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd 
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data
from torch.autograd import Variable
from sklearn.model_selection import train_test_split

In [2]:
train = pd.read_csv('/content/drive/MyDrive/fashion-mnist_train.csv')
print(train.shape)

(60000, 785)


In [3]:
test = pd.read_csv('/content/drive/MyDrive/fashion-mnist_test.csv')
print(test.shape)

(10000, 785)


In [4]:
y_train = train['label'].values
X_train = train.drop(['label'],1).values
y_test = test['label'].values
X_test = test.drop(['label'],1).values
print('y test shape : ',y_test.shape)
print('X test shape : ',X_test.shape)
print('y train shape : ',y_train.shape)
print('X train shape : ',X_train.shape)

y test shape :  (10000,)
X test shape :  (10000, 784)
y train shape :  (60000,)
X train shape :  (60000, 784)


In [5]:
BATCH_SIZE = 32


torch_X_train = torch.from_numpy(X_train).type(torch.LongTensor)
torch_y_train = torch.from_numpy(y_train).type(torch.LongTensor)

torch_X_test = torch.from_numpy(X_test).type(torch.LongTensor)
torch_y_test = torch.from_numpy(y_test).type(torch.LongTensor)

torch_X_train = torch_X_train.view(-1, 1,28,28).float()
torch_X_test = torch_X_test.view(-1,1,28,28).float()
print(torch_X_train.shape)
print(torch_X_test.shape)

train = torch.utils.data.TensorDataset(torch_X_train,torch_y_train)
test = torch.utils.data.TensorDataset(torch_X_test,torch_y_test)

train_loader = torch.utils.data.DataLoader(train, batch_size = BATCH_SIZE, shuffle = False)
test_loader = torch.utils.data.DataLoader(test, batch_size = BATCH_SIZE, shuffle = False)

torch.Size([60000, 1, 28, 28])
torch.Size([10000, 1, 28, 28])


In [9]:
def fit(model, train_loader):
    optimizer = torch.optim.SGD(model.parameters(),lr=0.001)
    error = nn.CrossEntropyLoss()
    EPOCHS = 5
    model.train()
    for epoch in range(EPOCHS):
        correct = 0
        for batch_idx, (X_batch, y_batch) in enumerate(train_loader):
            var_X_batch = Variable(X_batch).float()
            var_y_batch = Variable(y_batch)
            optimizer.zero_grad()
            output = model(var_X_batch)
            loss = error(output, var_y_batch)
            loss.backward()
            optimizer.step()

         
            predicted = torch.max(output.data, 1)[1] 
            correct += (predicted == var_y_batch).sum()
      
            if batch_idx % 50 == 0:
                print('Epoch : {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\t Accuracy:{:.3f}%'.format(
                    epoch, batch_idx*len(X_batch), len(train_loader.dataset), 100.*batch_idx / len(train_loader), loss.item(), float(correct*100) / float(BATCH_SIZE*(batch_idx+1))))

In [10]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=5)
        self.conv2 = nn.Conv2d(32, 32, kernel_size=5)
        self.conv3 = nn.Conv2d(32,64, kernel_size=5)
        self.fc1 = nn.Linear(3*3*64, 256)
        self.fc2 = nn.Linear(256, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(F.max_pool2d(self.conv2(x), 2))
        x = F.dropout(x, p=0.2, training=self.training)
        x = F.relu(F.max_pool2d(self.conv3(x),2))
        x = F.dropout(x, p=0.3, training=self.training)
        x = x.view(-1,3*3*64 )
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)
 
cnn = CNN()
print(cnn)

it = iter(train_loader)
X_batch, y_batch = next(it)
print(cnn.forward(X_batch).shape)

CNN(
  (conv1): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(32, 32, kernel_size=(5, 5), stride=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=10, bias=True)
)
torch.Size([32, 10])


In [11]:
fit(cnn,train_loader)

Epoch : 0 [0/60000 (0%)]	Loss: 11.650576	 Accuracy:12.500%
Epoch : 0 [1600/60000 (3%)]	Loss: 2.012340	 Accuracy:21.998%
Epoch : 0 [3200/60000 (5%)]	Loss: 1.584907	 Accuracy:30.229%
Epoch : 0 [4800/60000 (8%)]	Loss: 1.083411	 Accuracy:35.161%
Epoch : 0 [6400/60000 (11%)]	Loss: 1.403701	 Accuracy:39.583%
Epoch : 0 [8000/60000 (13%)]	Loss: 1.505786	 Accuracy:43.053%
Epoch : 0 [9600/60000 (16%)]	Loss: 1.037243	 Accuracy:45.909%
Epoch : 0 [11200/60000 (19%)]	Loss: 0.999852	 Accuracy:48.486%
Epoch : 0 [12800/60000 (21%)]	Loss: 1.174082	 Accuracy:50.234%
Epoch : 0 [14400/60000 (24%)]	Loss: 1.010363	 Accuracy:51.642%
Epoch : 0 [16000/60000 (27%)]	Loss: 1.056965	 Accuracy:53.050%
Epoch : 0 [17600/60000 (29%)]	Loss: 0.908825	 Accuracy:54.254%
Epoch : 0 [19200/60000 (32%)]	Loss: 0.782465	 Accuracy:55.226%
Epoch : 0 [20800/60000 (35%)]	Loss: 1.012701	 Accuracy:56.135%
Epoch : 0 [22400/60000 (37%)]	Loss: 0.833878	 Accuracy:56.959%
Epoch : 0 [24000/60000 (40%)]	Loss: 0.863376	 Accuracy:57.752%
Epoch

In [14]:
def evaluate(model):
    correct = 0 
    for test_imgs, test_labels in test_loader:
        test_imgs = Variable(test_imgs).float()
        output = model(test_imgs)
        predicted = torch.max(output,1)[1]
        correct += (predicted == test_labels).sum()
    print("Test accuracy:{:.3f} ".format( float(correct) / (len(test_loader)*BATCH_SIZE)))

In [15]:
evaluate(cnn)

Test accuracy:0.813 
